In [1]:
from google.colab import drive
drive.mount('/content/gdrive')  #mounting

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/My Drive/DEEP/DeepXi'  
# set working directory

/content/gdrive/My Drive/DEEP/DeepXi


In [3]:
ls  # view available files

1-s2.0-S0167639318304308-main.pdf  fig_training_example.png  run_demand_vb.sh
data/                              fig_xi_dist.png           run.m
DeepMMSE.pdf                       LICENSE.txt               run.sh
deepxi/                            log/                      seq_mask_batch.mat
eval_example.m                     main.py                   set/
eval_stats.m                       model/                    xi_bar_batch.mat
fig_Deep-Xi-ResNet.png             out/                      x_STMS_batch.mat
fig_front-end.png                  README.md
fig_tcn.gif                        requirements.txt


In [4]:
!pip install soundfile  # required installations

In [0]:
# required imports

import glob, os
import numpy as np
from scipy.io.wavfile import read
import soundfile as sf

# creating required math operations functions and directory functions

def save_wav(path, wav, f_s): 
    wav = np.squeeze(wav) 
    if isinstance(wav[0], np.float32): wav = np.asarray(np.multiply(wav, 32768.0), dtype=np.int16)
    sf.write(path, wav, f_s)

def read_wav(path):
    wav, f_s = sf.read(path, dtype='int16')
    return wav, f_s

def Batch(fdir, snr_l=[]):
    fname_l = [] # list of file names.
    wav_l = [] # list for waveforms.
    snr_test_l = [] # list of SNR levels for the test set. # signal to noise ratio
    fnames = ['*.wav', '*.flac', '*.mp3']

   #In Python, the glob module is used to retrieve files/pathnames matching a specified pattern.
    for fname in fnames:  # calling all values inside fnames
        for fpath in glob.glob(os.path.join(fdir, fname)): # calling directory path using gloab and os
            for snr in snr_l: # calling values in Snr_l
                if fpath.find('_' + str(snr) + 'dB') != -1:   # only if its not -1
                    snr_test_l.append(snr) #append the outcome form the loop after successfull iteration 
            (wav, _) = read_wav(fpath) # read back 
            if np.isnan(wav).any() or np.isinf(wav).any():  # if any way empty or infinity then raise error
                raise ValueError('Error: NaN or Inf value. File path: %s.' % (file_path))
            wav_l.append(wav) # else append it 
            fname_l.append(os.path.basename(os.path.splitext(fpath)[0])) # append respective path 

    len_l = [] # declare empty list
    maxlen = max(len(wav) for wav in wav_l) # get length of wav  present under wav_l
    wav_np = np.zeros([len(wav_l), maxlen], np.int16) # create numpy of zeros with same length

    for (i, wav) in zip(range(len(wav_l)), wav_l):  #defined loop
        wav_np[i,:len(wav)] = wav                          #only wav one to extract
        len_l.append(len(wav))    # appending outcomes
    return wav_np, np.array(len_l, np.int32), np.array(snr_test_l, np.int32), fname_l  # taking following returns


In [0]:
#CUSTOM CLASS FOR DIGITAL SIGNAL PROCESSING 





var = 0
varout = 0

class STFT:
    """
    Short-Term Fourier Transform:
        N_d                -  window duration (samples)
        N_s                -  window shift (samples)
        NFFT               -  number of DFT componts [ Discrete fourier transform]
        f_s                - sampling freq
    """
    #Mohit
    def __init__(self, N_d, N_s, NFFT, f_s):
        self.N_d = N_d
        self.N_s = N_s
        self.NFFT = NFFT
        self.f_s = f_s
        self.W = functools.partial(window_ops.hamming_window, periodic=False) # A callable that takes a window length and returns a [window_length] Tensor of samples in the provided datatype.
        self.ten = tf.cast(10.0, tf.float32) # Casting the tensor to the float32 type
    #Mohit
    import numpy as np
    import numpy.fft as fft

    def istft(stftArr, Nwin):
      Nwin=len(stftArr[1])-1
      arr = fft.irfft(stftArr)[:, :Nwin]
      return np.reshape(arr, -1)



    def polar_analysis(self, x):
        """
        x                  -  Input numpy array
        tf.signal.stft     -  Computes the Short-time Fourier Transform of signals
        Returns:
        tf.abs             -  computes absolute value of tensor         
        tf.math.angle      -  returns element wise arguments of a complex tensor
      """ 


        STFT = tf.signal.stft(x, self.N_d, self.N_s, self.NFFT, window_fn=self.W, pad_end=True) # Find STFT of a given signal
        return tf.abs(STFT), tf.math.angle(STFT) # Returns magnitude and phase angle of resulting STFT
    #Rohan
    def polar_synthesis(self, STMS, STPS):
        """
        tf.cast                      - Casts a tensor to a new type
        tf.complex                   - A Tensor of type complex64 or complex128
        tf.exp                       - performs exponential operation on a tensor
        
        Returns:
        tf.signal.inverse_stft       - inverse the stft input signals
        STMS                         - short time magnitude spectrum
        STPS                         - short time phase spectrum
        """
        STFT = tf.cast(STMS, tf.complex64)*tf.exp(1j*tf.cast(STPS, tf.complex64))
        return istft(STFT,self.NFFT) 
        #return tf.signal.inverse_stft(STFT, self.N_d, self.N_s, self.NFFT, tf.signal.inverse_stft_window_fn(self.N_s, self.W))

class DeepXiInput(STFT):
    def __init__(self, N_d, N_s, NFFT, f_s, mu=None, sigma=None):
        """
        defining mu and sigma
        """
        super().__init__(N_d, N_s, NFFT, f_s)
        self.mu = mu 
        self.sigma = sigma
    #Nobody        
    def observation(self, x):
        """
        Returns STMS and STPS from the given input numpy (converted audio file)
        """
        x = self.normalise(x)
        x_STMS, x_STPS = self.polar_analysis(x)
        return x_STMS, x_STPS
    


    # Defining all functions required for processing audio file after converting it to tensors using tensor mathematical operations. 
    #Shubham
    def normalise(self, x):
        #normailzation / standardization
        """
        Normalize the given input np array  
        """
        return tf.truediv(tf.cast(x, tf.float32), 32768.0) # Divides x tensor by y elementwise
    #Shubham
    def n_frames(self, N):
        """
        tf.math.ceil - Return the ceiling of the input, element-wise
        """
        return tf.cast(tf.math.ceil(tf.truediv(tf.cast(N, tf.float32), tf.cast(self.N_s, tf.float32))), tf.int32)


    #Surya
    def xi_hat(self, xi_bar_hat):
        """
        scipy.special.erfinv(y)  -  Inverse of the gause error function erf.
        """ 
        xi_db_hat = np.add(np.multiply(np.multiply(self.sigma, np.sqrt(2.0)),
                                       spsp.erfinv(np.subtract(np.multiply(2.0, xi_bar_hat), 1))), self.mu)
        return np.power(10.0, np.divide(xi_db_hat, 10.0))

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Add, Conv1D, Conv2D, Dense, Dropout,Flatten, LayerNormalization, MaxPooling2D, ReLU
import numpy as np

# define resnet, adding all parameters and then following layers.

class ResNet:
    def __init__(self,inp,n_outp,n_blocks,d_model,d_f,k,max_d_rate,padding,):
        self.d_model = d_model
        self.d_f = d_f
        self.k = k
        self.n_outp = n_outp
        self.padding = padding
        self.first_layer = self.feedforward(inp)
        self.layer_list = [self.first_layer]
        for i in range(n_blocks): self.layer_list.append(self.block(self.layer_list[-1], int(2**(i%(np.log2(max_d_rate)+1)))))
        self.logits = Conv1D(self.n_outp, 1, dilation_rate=1, use_bias=True)(self.layer_list[-1])
        self.outp = Activation('sigmoid')(self.logits)

    # 1st layer
    def feedforward(self, inp):
        ff = Conv1D(self.d_model, 1, dilation_rate=1, use_bias=False)(inp)
        norm = LayerNormalization(axis=2, epsilon=1e-6)(ff)
        act = ReLU()(norm)
        return act

    # 2nd layer
    def block(self, inp, d_rate):
        self.conv_1 = self.unit(inp, self.d_f, 1, 1, False)
        self.conv_2 = self.unit(self.conv_1, self.d_f, self.k, d_rate,
            False)
        self.conv_3 = self.unit(self.conv_2, self.d_model, 1, 1, True)
        residual = Add()([inp, self.conv_3])
        return residual

    # 3rd layer
    def unit(self, inp, n_filt, k, d_rate, use_bias):
        norm = LayerNormalization(axis=2, epsilon=1e-6)(inp)
        act = ReLU()(norm)
        conv = Conv1D(n_filt, k, padding=self.padding, dilation_rate=d_rate,
            use_bias=use_bias)(act)
        return conv


In [0]:
import numpy as np
from scipy.special import exp1

# loss funcations defining 

def mmse_lsa(xi, gamma):
    nu = np.multiply(np.divide(xi, np.add(1, xi)), gamma)
    return np.multiply(np.divide(xi, np.add(1, xi)), np.exp(np.multiply(0.5, exp1(nu)))) # MMSE-LSA gain function.

def gfunc(xi, gamma=None):
    G = mmse_lsa(xi, gamma)
    return G


In [0]:
from tensorflow.keras.layers import Input, Masking
from tensorflow.keras.models import Model
from tqdm import tqdm  # Instantly make your loops show a smart progress meter.
import math, os
import numpy as np
import tensorflow as tf

#main body, adding all parameters required

class DeepXi(DeepXiInput):
    def __init__(self,N_d,N_s,NFFT,f_s,ver="resnet-1.0c",**kwargs):
        super().__init__(N_d, N_s, NFFT, f_s)
        self.ver=ver
        self.n_feat = math.ceil(self.NFFT/2 + 1)
        self.n_outp = self.n_feat
        self.inp = Input(name='inp', shape=[None, self.n_feat], dtype='float32')
        self.mask = Masking(mask_value=0.0)(self.inp)
        self.network = ResNet(
            inp=self.mask,
            n_outp=self.n_outp,
            n_blocks=kwargs['n_blocks'],
            d_model=kwargs['d_model'],
            d_f=kwargs['d_f'],
            k=kwargs['k'],
            max_d_rate=kwargs['max_d_rate'],
            padding=kwargs['padding'],
            )
        self.model = Model(inputs=self.inp, outputs=self.network.outp)
        self.model.summary()
        if not os.path.exists("log/summary"):
            os.makedirs("log/summary")
        with open("log/summary/"+ self.ver + ".txt", "w") as f:
            self.model.summary(print_fn=lambda x: f.write(x + '\n'))

            
    
    # getting model summary out

    #under infer all required maths operations are done using tensor operations.

    def infer(self,test_x,test_x_len,test_x_base_names,model_path='model/saved_model/',out_type='y',out_path='out/denoised/',stats_path=None,n_filters=40,):
        
        out_path = out_path
        self.sample_stats(stats_path)
        self.model.load_weights(model_path +
            'variables/variables' )

        print("Processing observations...")
        print('test_x and len')
        print(test_x)
        print(test_x_len)
        x_STMS_batch, x_STPS_batch, n_frames = self.observation_batch(test_x, test_x_len)
        print('xstms batch and xstps batch and frames')
        print(x_STMS_batch)
        print(x_STPS_batch)
        print(n_frames)
        print("Performing inference...")
        xi_bar_hat_batch = self.model.predict(x_STMS_batch, batch_size=1, verbose=1)
        print(xi_bar_hat_batch)

        print("Performing synthesis...")
        batch_size = len(test_x_len)
        for i in tqdm(range(batch_size)):
            base_name = test_x_base_names[i]
            x_STMS = x_STMS_batch[i,:n_frames[i],:]
            x_STPS = x_STPS_batch[i,:n_frames[i],:]
            xi_bar_hat = xi_bar_hat_batch[i,:n_frames[i],:]
            xi_hat = self.xi_hat(xi_bar_hat)
            
            y_STMS = np.multiply(x_STMS, gfunc(xi_hat, xi_hat+1))
            y = self.polar_synthesis(y_STMS, x_STPS).numpy()
            save_wav(out_path+ base_name + '.wav', y, self.f_s)
            
    def sample_stats(self,stats_path='data/',sample_size=1000,train_s_list=None,train_d_list=None):
        if os.path.exists(stats_path + 'stats.npz'):
            print('Loading sample statistics...')
            with np.load(stats_path + 'stats.npz') as stats:
                self.mu = stats['mu_hat']
                self.sigma = stats['sigma_hat']
        elif train_s_list == None:
            raise ValueError('No stats.npz file exists. data/stats.p is available here: https://github.com/anicolson/DeepXi/blob/master/data/stats.npz.')

    def observation_batch(self, x_batch, x_batch_len):
        batch_size = len(x_batch)
        max_n_frames = self.n_frames(max(x_batch_len))
        x_STMS_batch = np.zeros([batch_size, max_n_frames, self.n_feat], np.float32)
        x_STPS_batch = np.zeros([batch_size, max_n_frames, self.n_feat], np.float32)
        n_frames_batch = [self. n_frames(i) for i in x_batch_len]
        for j in tqdm(range(batch_size)):
            i=0
            x_STMS, x_STPS = self.observation(x_batch[i,:x_batch_len[i]])
            x_STMS_batch[i,:n_frames_batch[i],:] = x_STMS.numpy()
            x_STPS_batch[i,:n_frames_batch[i],:] = x_STPS.numpy()
        return x_STMS_batch, x_STPS_batch, n_frames_batch

    

In [37]:
#VARIABLES FOR THE MODEL

d_model  = 256
n_blocks  = 40
d_f = 64
k =  3
max_d_rate = 16 
padding = "causal"
ver="resnet-1.0c"
f_s  = 16000 
T_d  = 32 
T_s  =  16 
causal  =   1
ver="resnet-1.0c"

# VARIABLES FOR INFERENCE
data_path='data/' # Path of the sample_stats file that to be loaded for inference purpose.
test_x_path='set/test_noisy_speech' # Path of the inputs : noisy audio files
out_path='out/denoised/' # Path to which out the output audio file is saved.
model_path='model/saved_model/' # Path of the TF Saved_model



N_d = int(f_s*T_d*0.001) # window duration (samples).
N_s = int(f_s*T_s*0.001) # window shift (samples).
NFFT = int(pow(2, np.ceil(np.log2(N_d)))) # number of DFT components.

# set test

test_x, test_x_len, _, test_x_base_names = Batch(test_x_path)

# run the model

deepxi = DeepXi(N_d=N_d,N_s=N_s,NFFT=NFFT,f_s=f_s,d_model=d_model,n_blocks=n_blocks,d_f=d_f,k=k,max_d_rate =max_d_rate,padding=padding,
  causal=causal,ver= ver)




Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp (InputLayer)                [(None, None, 257)]  0                                            
__________________________________________________________________________________________________
masking_4 (Masking)             (None, None, 257)    0           inp[0][0]                        
__________________________________________________________________________________________________
conv1d_488 (Conv1D)             (None, None, 256)    65792       masking_4[0][0]                  
__________________________________________________________________________________________________
layer_normalization_484 (LayerN (None, None, 256)    512         conv1d_488[0][0]                 
____________________________________________________________________________________________

In [38]:
# once model is been load we can start infering directly from the below lines.
import scipy.special as spsp


deepxi.infer(test_x=test_x,test_x_len=test_x_len,test_x_base_names=test_x_base_names,
        model_path=model_path,out_path= out_path,stats_path= data_path)

Loading sample statistics...


100%|██████████| 1/1 [00:00<00:00, 17.36it/s]

Processing observations...
test_x and len
[[  0   0   0 ... 192 215 228]]
[178277]
xstms batch and xstps batch and frames
[[[3.5414509e-02 1.0582294e-01 1.5940690e-01 ... 1.2714321e-04
   2.0921722e-04 1.3193674e-04]
  [1.8737470e-01 4.8111248e-01 6.9399112e-01 ... 2.1845955e-04
   2.4234368e-04 2.1667778e-04]
  [1.2425509e-01 3.9077184e-01 5.7660270e-01 ... 5.6300356e-05
   9.1046080e-05 1.8748268e-04]
  ...
  [1.5776178e-01 4.2924857e-01 3.8987401e-01 ... 2.7216316e-04
   2.1874612e-04 2.4438649e-04]
  [1.9885100e-01 3.7204039e-01 4.3069333e-01 ... 4.2559630e-03
   4.0232679e-03 3.9512366e-03]
  [6.8450905e-02 6.4667836e-02 5.4556489e-02 ... 2.0935086e-03
   2.3127752e-03 2.4365112e-03]]]
[[[ 0.          2.606935   -2.1490455  ... -2.4505608   2.4742794
    3.1415927 ]
  [ 0.          1.5957347  -1.8102022  ...  0.32534197 -0.4740404
    0.        ]
  [ 0.          0.1850563  -3.0245085  ... -0.5151      1.3295093
    0.        ]
  ...
  [ 0.         -3.0979102   0.4019953  ... -0.20

1/1 [==============================] - 0s 1ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

[[[0.22427684 0.23503181 0.21198487 ... 0.39194036 0.39773834 0.40324825]
  [0.35561317 0.2063398  0.17556733 ... 0.46800455 0.46870983 0.454349  ]
  [0.38557103 0.23427504 0.2563734  ... 0.52110463 0.52190244 0.5092561 ]
  ...
  [0.24578765 0.15624347 0.18187404 ... 0.587378   0.59688765 0.5636915 ]
  [0.30538982 0.19252211 0.2520641  ... 0.82325673 0.8313161  0.7991761 ]
  [0.25676832 0.18592101 0.22084874 ... 0.26071543 0.25195175 0.276119  ]]]
Performing synthesis...


NameError: ignored

In [0]:
# breakdown of above model operations.
# all operations are need.
# operations done are maths operations. This operations are been performed using tensor.


import math 
from tqdm import tqdm  # Instantly make your loops show a smart progress meter.

def Batch(fdir, snr_l=[]):
    fname_l = [] # list of file names.
    wav_l = [] # list for waveforms.
    snr_test_l = [] # list of SNR levels for the test set. # signal to noise ratio
    fnames = ['*.wav', '*.flac', '*.mp3']

#In Python, the glob module is used to retrieve files/pathnames matching a specified pattern.
    for fname in fnames:  # calling all values inside fnames
        for fpath in glob.glob(os.path.join(fdir, fname)): # calling directory path using gloab and os
            for snr in snr_l: # calling values in Snr_l
                if fpath.find('_' + str(snr) + 'dB') != -1:   # only if its not -1
                    snr_test_l.append(snr) 
            (wav, _) = read_wav(fpath) # read back 
            if np.isnan(wav).any() or np.isinf(wav).any():  # if any way empty or infinity then raise error
                raise ValueError('Error: NaN or Inf value. File path: %s.' % (file_path))
            wav_l.append(wav) # else append it 
            fname_l.append(os.path.basename(os.path.splitext(fpath)[0]))

    len_l = [] 
    maxlen = max(len(wav) for wav in wav_l) # get length of wav  present under wav_l
    wav_np = np.zeros([len(wav_l), maxlen], np.int16) # create numpy of zeros with same length

    for (i, wav) in zip(range(len(wav_l)), wav_l):
        wav_np[i,:len(wav)] = wav
        len_l.append(len(wav)) 
    return wav_np, np.array(len_l, np.int32), np.array(snr_test_l, np.int32), fname_l

f_s  = 16000 
T_d  = 32 
T_s  =  16 
N_s = int(f_s*T_s*0.001)  
N_d = int(f_s*T_d*0.001)  

def n_frames(N):
        return tf.cast(tf.math.ceil(tf.truediv(tf.cast(N, tf.float32), tf.cast(N_s, tf.float32))), tf.int32)

NFFT = int(pow(2, np.ceil(np.log2(N_d))))
n_feat = math.ceil(NFFT/2 + 1)

import functools

W = functools.partial(window_ops.hamming_window, periodic=False)

import tensorflow as tf

def polar_analysis(x):
  STFT = tf.signal.stft(x, N_d, N_s, NFFT, window_fn=W, pad_end=True)
  return tf.abs(STFT), tf.math.angle(STFT)

def normalise(x):
  return tf.truediv(tf.cast(x, tf.float32), 32768.0)  

def observation(x):
  x = normalise(x)
  x_STMS, x_STPS = polar_analysis(x)
  return x_STMS, x_STPS

def observation_batch(x_batch, x_batch_len):
        print('\n')  
        batch_size = len(x_batch)
        print('batch_size',batch_size)
        print('\n')
        print(len(x_batch),'size xbatch')
        print('\n')
        print('x_batch',x_batch)
        print('\n')
        print('x_batch_len',x_batch_len)
        print('\n')
        max_n_frames = n_frames(max(x_batch_len))
        print('\n')
        print('max_n_frames',max_n_frames)
        x_STMS_batch = np.zeros([batch_size, max_n_frames, n_feat], np.float32)
        print('\n')
        print('x_STMS_batch',x_STMS_batch)
        print('x_STMS_batch_len',len(x_STMS_batch[0]))
        x_STPS_batch = np.zeros([batch_size, max_n_frames, n_feat], np.float32)
        print('\n')
        print('x_STPS_batch',x_STPS_batch)
        n_frames_batch = [n_frames(i) for i in x_batch_len]
        print('n_frames_batch',n_frames_batch)
        for i in tqdm(range(batch_size)):
            x_STMS, x_STPS = observation(x_batch[i,:x_batch_len[i]])
            print('x_STMS',x_STMS)
            print('NEXT','\n')
            print('x_STPS',x_STPS)
            x_STMS_batch[i,:n_frames_batch[i],:] = x_STMS.numpy()
            x_STPS_batch[i,:n_frames_batch[i],:] = x_STPS.numpy()
            print('\n')
            print('x_STMS_batch',x_STMS_batch)
            print('\n')
            print('x_STPS_batch',x_STMS_batch)
            print('\n')
            print('n_frames_batch',n_frames_batch)
            print('\n')

        return x_STMS_batch, x_STPS_batch, n_frames_batch



In [0]:
a=[178277]

In [0]:
a[0]

In [0]:
A=observation_batch(test_x,test_x_len)

100%|██████████| 1/1 [00:00<00:00, 17.67it/s]



batch_size 1


1 size xbatch


x_batch [[  0   0   0 ... 192 215 228]]


x_batch_len [178277]




max_n_frames tf.Tensor(697, shape=(), dtype=int32)


x_STMS_batch [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
x_STMS_batch_len 697


x_STPS_batch [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
n_frames_batch [<tf.Tensor: shape=(), dtype=int32, numpy=697>]
x_STMS tf.Tensor(
[[3.5414509e-02 1.0582294e-01 1.5940690e-01 ... 1.2714321e-04
  2.0921722e-04 1.3193674e-04]
 [1.8737470e-01 4.8111248e-01 6.9399112e-01 ... 2.1845955e-04
  2.4234368e-04 2.1667778e-04]
 [1.2425509e-01 3.9077184e-01 5.7660270e-01 ... 5.6300356e-05
  9.1046080e-05 1.8748268e-04]
 ...
 [1.5776178e-01 4.2924857e-01 3.8987401e-01 ... 2.7216316e-04
  2.1874612e-04 2.4438649e-04]
 [1.9885100e

In [0]:
test_x_path='set/test_noisy_speech'
test_x, test_x_len, _, test_x_base_names = Batch(test_x_path)




In [0]:
from tqdm import tqdm

for i in tqdm(range(5)):
  pass

100%|██████████| 5/5 [00:00<00:00, 5344.42it/s]


In [0]:
for i in range(1):
  print(i)

0
